In [66]:
# # Versão da Linguagem Python
# from platform import python_version
# print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [67]:
# Importar as bibliotecas

import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import optimize
import numpy_financial as npf
import os

In [68]:
# Versões dos pacotes usados neste jupyter notebook
# %reload_ext watermark
# %watermark -a "GEPRE" --iversions

In [69]:
# os.chdir('//sad154000/Corporativo/Gepre/Estagiarios/Validacao_Precificacao/20231113')
# os.chdir('//sad154000/Corporativo/Gepre/Estagiarios/PRECIFICACAO/outubro/02102024')

In [70]:
# Importar planilha do excel
base_dados = 'Base de Dados - Taxi Verde.xlsx'

# Importar base de produtos 
sheet1 =  pd.read_excel(base_dados, sheet_name='produtos')

# Importar curva DI/Funding
sheet2 =  pd.read_excel(base_dados, sheet_name='curva')

In [71]:
# Quantidade de Produtos
rows = len(sheet1.axes[0])

#Valores Tributos
#IR/CSLL
ir = 0.45

# PIS/COfins
pis_cofins = 0.0465

# PIS/COFINS com ISS - Aqui o PIS/COFINS com ISS é igual ao PIS/COFINS. Trata-se de um ajuste necessário para o projeto 'Táxi Verde'
iss = 0.0465

indexador = sheet1.index.values

Margem_Liquida = pd.Series(index = indexador, dtype='float64')
Taxa_ML = pd.Series(index = indexador, dtype='float64')
Tarifa_ML = pd.Series(index = indexador, dtype='float64')

In [72]:

def ML_tarifa(tarifa_var):

    
    # Cálculos iniciais (iguais ao original)
    data_inicial = sheet1.at[k,'Data inicial:']
    tabela_calculo = sheet2.loc[sheet2['Data']>=data_inicial]
    tabela_calculo.insert(0,'Período',range(0,len(tabela_calculo)))
    prazo = sheet1.at[k,'Prazo:']
    tabela_calculo = tabela_calculo.loc[tabela_calculo['Período']<=prazo]
    tabela_calculo.reset_index(inplace = True)
    #curva DI para o período
    
    #tipo de planilha: Price, Limite ou Antecipação
    plan = sheet1.at[k,'Plan:']
    #curva DI ao ano e ao mês
    di_ano=tabela_calculo['Curva']
    di_mes=(di_ano+1)**(1/12)-1
    #cálculo da taxa de desconto
    desconto = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    desconto[0] = 1
    #for i in range(0,di_mes.shape[0]):
    for i in range(1,prazo+1):    
        desconto[i] = 1/((1+di_mes[i])*(1/desconto[i-1]))
       
    
    #inserindo a Taxa de Desconto e as curvas em tabela única        
    curvas = pd.DataFrame(desconto)
    curvas.columns=['Fator de desconto']
    curvas.insert(0,'DI ano',di_ano,True)
    curvas.insert(1,'DI mês',di_mes,True)
    tabela_calculo.rename(columns = {'Curva':'DI ano'}, inplace = True)
    tabela_calculo.insert(3,'DI mês',di_mes)
    tabela_calculo.insert(4,'Taxa de Desconto',desconto)
    
    #cálculo da taxa Pré ou Pós
    tipo_taxa = sheet1.at[k,'Tipo de Taxa:']
    taxa_nominal = (sheet1.at[k,'Taxa Nominal a.m.:'])/100
    taxa_efetiva =  pd.DataFrame(index = tabela_calculo['Período'],dtype='float64') 
    indexador = sheet1.at[k,'% CDI (indexador):']
    taxa_efetiva = np.where((tipo_taxa=='Pre') | (tipo_taxa == 'Pré'),taxa_nominal,((1+taxa_nominal)*(1+di_mes*indexador))-1)  
    
    ### motor de cálculo
    
    juros = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    juros[0]= 0
    
    #Saldo Devedor Inicial = Ticket Médio
    
    sd_inicial = sheet1.at[k,'Ticket Médio (R$):']
    
    saldo_devedor = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    saldo_devedor[0] = sd_inicial
    
    parcela = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    parcela[0] = 0
    
    amortizacao = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    amortizacao[0] = 0
    
    captacao = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    captacao[0] = 0
    
    perda_esperada = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    
    #Tarifa
    tarifa= sheet1.at[k,'Tarifa (R$):']
    #receita de relacionamento
    r_relacionamento = sheet1.at[k,'Receita de Relacionamento (R$):']
    #Custo de Liberação
    c_liberacao = sheet1.at[k,'Custo de Liberação (R$):']
    #Custo de Manutenção
    c_manutencao = sheet1.at[k,'Custo de Manutenção (R$):']
    #Custo de Relacionamento
    c_relacionamento = sheet1.at[k,'Custo de Relacionamento (R$):']
    #Custo com Correspondente
    c_correspondente = sheet1.at[k,'Custo com Correspondente(%):']
    
    r_operacional = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    c_operacional = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    r_operacional[0]=tarifa_var
    c_operacional[0]=-(sd_inicial*c_correspondente + c_liberacao)
    
    #Tributos PIS/COFINS/ISS
    pis_cofins_iss = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    pis_cofins_iss[0] = -((captacao[0]+juros[0])*pis_cofins+(r_operacional[0]*iss))
    
    lair = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    lair[0] = (juros[0] + captacao[0] + perda_esperada[0] + c_operacional[0] + r_operacional[0] + pis_cofins_iss[0])
    
    imposto_renda = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    imposto_renda[0] = -lair[0]*ir
    
    lucro = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    lucro[0] = lair[0]+imposto_renda[0]
    
    #curva de funding
    funding = di_mes
    
    
    carência = sheet1.at[k,'Carência (meses):']
    for i in range(1,prazo+1):
        #cálculo dos juros e da captação   
        juros[i] = saldo_devedor[i-1]*taxa_efetiva[i]
        captacao[i]= -saldo_devedor[i-1]*funding[i]
        #Período de carência
        if i <= carência:
            if plan == 'Antecipações':
                parcela[i]=0
                amortizacao[i]=0
                saldo_devedor[i]=saldo_devedor[i-1]+juros[i]
            else:
                parcela[i]=juros[i]
                amortizacao[i]=0
                saldo_devedor[i]=saldo_devedor[i-1]+juros[i]-parcela[i]
        else:
          #Sistema Price
            if plan == 'Price':
                parcela[i]=npf.pmt(taxa_efetiva[i], prazo-tabela_calculo['Período'][i-1], -saldo_devedor[i-1])
                amortizacao[i]=parcela[i]-juros[i]
                saldo_devedor[i]=saldo_devedor[i-1]+juros[i]-parcela[i]
            elif plan == 'Limites':
         #Sistema Limites
                if i == prazo:
                    amortizacao[i]= sd_inicial
                    saldo_devedor[i]=saldo_devedor[i-1]-amortizacao[i]
                    parcela[i]= amortizacao[i]+juros[i]
                else:
                    amortizacao[i]=0
                    parcela[i] = 0
                    saldo_devedor[i]=saldo_devedor[i-1]
            elif plan == 'Antecipações':
         #Sistema Antecipações  
                 if i == prazo:
                    amortizacao[i]= saldo_devedor[i-1]
                    saldo_devedor[i]=saldo_devedor[i-1]-amortizacao[i]
                    parcela[i]= amortizacao[i]+juros[i]
                 else:
                    amortizacao[i]=0
                    parcela[i] = 0
                    saldo_devedor[i]=saldo_devedor[i-1] +juros[i] 
                
        #calculo da perda esperada
        perda_esperada[i] = -(parcela[i]*sheet1.at[k,'Perda Esperada (%):'])
        
        #calculo da Receita e do Custo Operacional
        
        r_operacional[i]= r_relacionamento
        c_operacional[i]=-(c_manutencao+c_relacionamento)
        
        #calculo pis cofins iss
        pis_cofins_iss[i]= -((captacao[i]+juros[i])*pis_cofins+(r_operacional[i]*iss))
        
        #Cálculo do LAIR e IR
        lair[i]= juros[i] + captacao[i] + perda_esperada[i] + c_operacional[i] + r_operacional[i] + pis_cofins_iss[i]
        
        imposto_renda[i] = lair[i]*ir
        
        lucro[i] = lair[i] - imposto_renda[i]
    
        #valor presente lucro
        vp_lucro = lucro*desconto
        vp_lucro = vp_lucro.sum()
        #valor presente saldo devedor
        vp_sd = saldo_devedor*desconto
        vp_sd = vp_sd.sum()
    
        #Calculo Margem Líquida
        margem_liquida = (vp_lucro/vp_sd)*12
    ml = margem_liquida - sheet1.at[k,'ML (% a.a.):']
    print("ML:", ml)
    return ml

In [73]:
for k in tqdm(range(0,rows)):
    #cálculo da margem líquida
    #data inicial de calculo
    data_inicial = sheet1.at[k,'Data inicial:']
     
    tabela_calculo = sheet2.loc[sheet2['Data']>=data_inicial]
    #incluir período
    tabela_calculo.insert(0,'Período',range(0,len(tabela_calculo)))
    ##prazo da operação
    prazo = sheet1.at[k,'Prazo:']
    tabela_calculo = tabela_calculo.loc[tabela_calculo['Período']<=prazo]
    tabela_calculo.reset_index(inplace = True)
    #curva DI para o período
    
    #tipo de planilha: Price, Limite ou Antecipação
    plan = sheet1.at[k,'Plan:']
    #curva DI ao ano e ao mês
    di_ano=tabela_calculo['Curva']
    di_mes=(di_ano+1)**(1/12)-1
    #cálculo da taxa de desconto
    desconto = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    desconto[0] = 1
    #for i in range(0,di_mes.shape[0]):
    for i in range(1,prazo+1):    
        desconto[i] = 1/((1+di_mes[i])*(1/desconto[i-1]))
       
    
    #inserindo a Taxa de Desconto e as curvas em tabela única        
    curvas = pd.DataFrame(desconto)
    curvas.columns=['Fator de desconto']
    curvas.insert(0,'DI ano',di_ano,True)
    curvas.insert(1,'DI mês',di_mes,True)
    tabela_calculo.rename(columns = {'Curva':'DI ano'}, inplace = True)
    tabela_calculo.insert(3,'DI mês',di_mes)
    tabela_calculo.insert(4,'Taxa de Desconto',desconto)
    
    #cálculo da taxa Pré ou Pós
    tipo_taxa = sheet1.at[k,'Tipo de Taxa:']
    taxa_nominal = (sheet1.at[k,'Taxa Nominal a.m.:'])/100
    taxa_efetiva =  pd.DataFrame(index = tabela_calculo['Período'],dtype='float64') 
    indexador = sheet1.at[k,'% CDI (indexador):']
    taxa_efetiva = np.where((tipo_taxa=='Pre') | (tipo_taxa == 'Pré'),taxa_nominal,((1+taxa_nominal)*(1+di_mes*indexador))-1)  
    
    ### motor de cálculo
    
    juros = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    juros[0]= 0
    
    #Saldo Devedor Inicial = Ticket Médio
    
    sd_inicial = sheet1.at[k,'Ticket Médio (R$):']
    
    saldo_devedor = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    saldo_devedor[0] = sd_inicial
    
    parcela = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    parcela[0] = 0
    
    amortizacao = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    amortizacao[0] = 0
    
    captacao = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    captacao[0] = 0
    
    perda_esperada = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    
    #Tarifa
    tarifa= sheet1.at[k,'Tarifa (R$):']
    #receita de relacionamento
    r_relacionamento = sheet1.at[k,'Receita de Relacionamento (R$):']
    #Custo de Liberação
    c_liberacao = sheet1.at[k,'Custo de Liberação (R$):']
    #Custo de Manutenção
    c_manutencao = sheet1.at[k,'Custo de Manutenção (R$):']
    #Custo de Relacionamento
    c_relacionamento = sheet1.at[k,'Custo de Relacionamento (R$):']
    #Custo com Correspondente
    c_correspondente = sheet1.at[k,'Custo com Correspondente(%):']
    
    r_operacional = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    c_operacional = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    r_operacional[0]=tarifa
    c_operacional[0]=-(sd_inicial*c_correspondente + c_liberacao)
    
    #Tributos PIS/COFINS/ISS
    pis_cofins_iss = pd.Series(0,index = tabela_calculo['Período'],dtype='float64')
    pis_cofins_iss[0] = -((captacao[0]+juros[0])*pis_cofins+(r_operacional[0]*iss))
    
    lair = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    lair[0] = (juros[0] + captacao[0] + perda_esperada[0] + c_operacional[0] + r_operacional[0] + pis_cofins_iss[0])
    
    imposto_renda = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    imposto_renda[0] = -lair[0]*ir
    
    lucro = pd.Series(index = tabela_calculo['Período'],dtype='float64')
    lucro[0] = lair[0]+imposto_renda[0]
    
    #curva de funding
    funding = di_mes
    
    
    carência = sheet1.at[k,'Carência (meses):']
    for i in range(1,prazo+1):
        #cálculo dos juros e da captação   
        juros[i] = saldo_devedor[i-1]*taxa_efetiva[i]
        captacao[i]= -saldo_devedor[i-1]*funding[i]
        #Período de carência
        if i <= carência:
            if plan == 'Antecipações':
                parcela[i]=0
                amortizacao[i]=0
                saldo_devedor[i]=saldo_devedor[i-1]+juros[i]
            else:
                parcela[i]=juros[i]
                amortizacao[i]=0
                saldo_devedor[i]=saldo_devedor[i-1]+juros[i]-parcela[i]
        else:
          #Sistema Price
            if plan == 'Price':
                parcela[i]=npf.pmt(taxa_efetiva[i], prazo-tabela_calculo['Período'][i-1], -saldo_devedor[i-1])
                amortizacao[i]=parcela[i]-juros[i]
                saldo_devedor[i]=saldo_devedor[i-1]+juros[i]-parcela[i]
            elif plan == 'Limites':
         #Sistema Limites
                if i == prazo:
                    amortizacao[i]= sd_inicial
                    saldo_devedor[i]=saldo_devedor[i-1]-amortizacao[i]
                    parcela[i]= amortizacao[i]+juros[i]
                else:
                    amortizacao[i]=0
                    parcela[i] = 0
                    saldo_devedor[i]=saldo_devedor[i-1]
            elif plan == 'Antecipações':
         #Sistema Antecipações  
                 if i == prazo:
                    amortizacao[i]= saldo_devedor[i-1]
                    saldo_devedor[i]=saldo_devedor[i-1]-amortizacao[i]
                    parcela[i]= amortizacao[i]+juros[i]
                 else:
                    amortizacao[i]=0
                    parcela[i] = 0
                    saldo_devedor[i]=saldo_devedor[i-1] +juros[i] 
                
        #calculo da perda esperada
        perda_esperada[i] = -(parcela[i]*sheet1.at[k,'Perda Esperada (%):'])
        
        #calculo da Receita e do Custo Operacional
        
        r_operacional[i]= r_relacionamento
        c_operacional[i]=-(c_manutencao+c_relacionamento)
        
        #calculo pis cofins iss
        pis_cofins_iss[i]= -((captacao[i]+juros[i])*pis_cofins+(r_operacional[i]*iss))
        
        #Cálculo do LAIR e IR
        lair[i]= juros[i] + captacao[i] + perda_esperada[i] + c_operacional[i] + r_operacional[i] + pis_cofins_iss[i]
        
        imposto_renda[i] = lair[i]*ir
        
        lucro[i] = lair[i] - imposto_renda[i]
    
        #valor presente lucro
        vp_lucro = lucro*desconto
        vp_lucro = vp_lucro.sum()
        #valor presente saldo devedor
        vp_sd = saldo_devedor*desconto
        vp_sd = vp_sd.sum()
        
        margem_liquida = 0.01
        Margem_Liquida[k] = margem_liquida
        tarifa_calculada = optimize.newton(ML_tarifa, sheet1.at[k,'Tarifa (R$):'])
        Tarifa_ML[k] = tarifa_calculada  

  0%|          | 0/22 [00:00<?, ?it/s]


RuntimeError: Tolerance of 3.211308000001736 reached. Failed to converge after 1 iterations, value is 32115.291308000003.

In [65]:
sheet1['ML'] = Margem_Liquida
sheet1['teste'] = Tarifa_ML
sheet1.to_excel('voumematarembreve2.xlsx', index = False)